In [3]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [4]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

Intitution: If we know the solution to subproblems $v_*(s')$

Then solution $v_*(s)$ can be found by one-step lookahead: $v_*(s) \leftarrow \max_{a \in \mathcal{A}} [\mathcal{R}_s^a + \gamma \sum_{s' \in \mathcal{S}} \mathcal{P}_{ss'}^a v_*(s')]$

Pipeline:

1. Initialize $V(s) = 0$ for all states $s \in S$.

2. Repeat until convergence:$\Delta \gets 0$
   For each state $s \in S$:
   $$
   v \gets V(s)
   $$
   $$
   V(s) \gets \max_a \sum_{s'} P(s' | s, a) \big[ R(s, a) + \gamma V(s') \big]
   $$
   $$
   \Delta \gets \max(\Delta, |v - V(s)|)
   $$
   Until $\Delta < \epsilon$.

3. For each state $s \in S$: $\pi^*(s) \gets \arg\max_a \sum_{s'} P(s' | s, a) \big[ R(s, a) + \gamma V^*(s') \big]$

4. Return $V^*(s), \pi^*(s)$.

In [8]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm to compute optimal policy and value function.
    
    Args:
        env: OpenAI environment. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is the number of states in the environment. 
            env.nA is the number of actions in the environment.
        theta: A small threshold for the change in value function to decide convergence.
        discount_factor: The gamma (discount factor) used in the Bellman equation for future rewards.
        
    Returns:
        policy: The optimal policy, a matrix of size (nS, nA), where policy[s, a] = 1 if action 'a' is optimal at state 's'.
        V: The optimal value function, a vector of length nS, containing the maximum expected rewards from each state.
    """

    def one_step_lookahead(state, V):
        """
        Helper function to compute the expected value for each action at the given state.
        
        Args:
            state: The current state (int).
            V: Current value function (vector of size env.nS).
        
        Returns:
            A vector of size env.nA containing the expected value for each action in the given state.
        """
        Vs = np.zeros(env.nA)  # Initialize the value array for each action in the current state
        
        # Loop over all actions
        for a in range(env.nA):  
            for prob, next_state, reward, done in env.P[state][a]:  
                # Bellman equation
                Vs[a] += prob * (reward + discount_factor * V[next_state])
                
        return Vs

    V = np.zeros(env.nS)  
    policy = np.zeros([env.nS, env.nA])
    
    iteration = 0
    while True:
        # Track the maximum change in value function during this iteration
        delta = 0  

        for s in range(env.nS):
            # One-step lookahead for the optimal value of each action
            Vs = one_step_lookahead(s, V)
            optimal_action_value = np.max(Vs) 

            # Compute the change in value function for this state
            delta = max(delta, np.abs(optimal_action_value - V[s]))
            
            # Update the value function for the state
            V[s] = optimal_action_value
        
        iteration += 1
        
        # Check for convergence
        if delta < theta:
            break
    
    # Determine the optimal policy by selecting the action with the highest value for each state
    for s in range(env.nS):
        Vs = one_step_lookahead(s, V)
        best_action = np.argmax(Vs)  
        policy[s, best_action] = 1.0  

    print(f"Optimal policy found after {iteration} iterations.") 
    return policy, V

In [9]:
policy, v = value_iteration(env)

print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Optimal policy found after 4 iterations.
Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [7]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)